Connecting to Google Drive 

In [1]:
def connect_to_drive():
  from google.colab import drive
  drive.mount('/content/gdrive', force_remount=True)

!pip install import_ipynb 
import import_ipynb 
connect_to_drive()
%cd '/content/gdrive/My Drive/Bioinformatics2' 

  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-py3-none-any.whl size=2975 sha256=8e48009d7bb42a8ca65d62d478ea0c3916895ea8e1fab10c3232c85078c2dafe
  Stored in directory: /root/.cache/pip/wheels/b1/5e/dc/79780689896a056199b0b9f24471e3ee184fbd816df355d5f0
Successfully built import-ipynb
Mounted at /content/gdrive
/content/gdrive/My Drive/Bioinformatics2


Importing modules

In [3]:
import torch
from utils import *
install_requirements()
from VascularDataset import *
import albumentations as A
from torchvision.transforms import *
from torch.utils.data import DataLoader
import segmentation_models_pytorch as smp
import albumentations
from albumentations.augmentations.functional import elastic_transform_approx
import segmentation_models_pytorch as smp
import wandb

     |████████████████████████████████| 87 kB 4.1 MB/s 
     |████████████████████████████████| 376 kB 20.2 MB/s 
     |████████████████████████████████| 58 kB 5.3 MB/s 
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.6.3-py3-none-any.whl size=12421 sha256=07c708940f5a7aef0ec3bc4cf2c53e560280c6a2216ee4c8a14aa2b20974f962
  Stored in directory: /root/.cache/pip/wheels/90/6b/0c/f0ad36d00310e65390b0d4c9218ae6250ac579c92540c9097a
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60965 sha256=87ac46bcb52fc914be027474cccb11c7ea2ed6b328cadfe1f9ad6f395f1c42a9
  Stored in directory: /root/.cache/pip/wheels/ed/27/e8/9543d42de2740d3544db96aefef63bda3f2c1761b3334f4873
Successfully built efficientnet-pytorch pretrainedmodels
     |████████████████████████████████| 1.7 MB 7.4 MB/s 
     |████████████████████████████████| 139 kB 64.3 MB/s 
     |████████████████████████████████| 180 kB 63.8 MB/s 
     |████████████████████████████████

Defining dataset and dataloaders

In [4]:
input_path_training='/content/gdrive/.shortcut-targets-by-id/1mtNVywqwmtCBefeGQrY2c6F6xBQQWq30/vascular_segmentation/Train'
input_path_test='/content/gdrive/.shortcut-targets-by-id/1mtNVywqwmtCBefeGQrY2c6F6xBQQWq30/vascular_segmentation/Test'

txt_with_training_paths_inputs='/content/gdrive/MyDrive/Bioinformatics2/dataset_paths/training_paths_inputs.txt'
txt_with_training_paths_targets='/content/gdrive/MyDrive/Bioinformatics2/dataset_paths/training_paths_targets.txt'
txt_with_validation_paths_inputs='/content/gdrive/MyDrive/Bioinformatics2/dataset_paths/validation_paths_inputs.txt'
txt_with_validation_paths_targets='/content/gdrive/MyDrive/Bioinformatics2/dataset_paths/validation_paths_targets.txt'
txt_with_test_paths_inputs='/content/gdrive/MyDrive/Bioinformatics2/dataset_paths/test_paths_inputs.txt'
txt_with_test_paths_targets='/content/gdrive/MyDrive/Bioinformatics2/dataset_paths/test_paths_targets.txt'

imgs_training_input_split=read_list_images_path_from_txt(txt_with_training_paths_inputs)[:10]
imgs_training_target_split=read_list_images_path_from_txt(txt_with_training_paths_targets)[:10]
imgs_validation_input_split=read_list_images_path_from_txt(txt_with_validation_paths_inputs)
imgs_validation_target_split=read_list_images_path_from_txt(txt_with_validation_paths_targets)
imgs_test_input=read_list_images_path_from_txt(txt_with_test_paths_inputs)
imgs_test_target=read_list_images_path_from_txt(txt_with_test_paths_targets)

Selecting transforms

In [5]:
from torchvision.transforms import *
import albumentations as A
from albumentations.augmentations.transforms import *


affine_transform=A.Compose([Flip(), ShiftScaleRotate()])
elastic_transform=A.Compose([ElasticTransform()])
pixel_wise_transform=A.Compose([GaussNoise(), HueSaturationValue(), RandomBrightnessContrast(), RandomGamma()])
training_dataset_plain=VascularDataset(imgs_training_input_split, imgs_training_target_split)
training_dataset_affine=VascularDataset(imgs_training_input_split, imgs_training_target_split, transform=affine_transform)
training_dataset_elastic=VascularDataset(imgs_training_input_split, imgs_training_target_split, transform=elastic_transform)
training_dataset_pixel_wise=VascularDataset(imgs_training_input_split, imgs_training_target_split, transform=pixel_wise_transform)

Visualizing predictions on training



In [7]:
#Compute mean and std without Imagenet
t=ToTensor()
mean=0
std=0
for path in imgs_training_input_split:
  im=Image.open(path)
  image=t(im)
  mean+=torch.mean(image, [1,2], keepdim=True).squeeze()
  std+=torch.std(image, [1,2], keepdim=True).squeeze()

print(mean/len(training_dataset_plain))
print(std/len(training_dataset_plain))

tensor([0.7160, 0.4878, 0.6433, 1.0000])
tensor([0.1680, 0.2098, 0.1685, 0.0000])


In [ ]:
#mean: (0.7160, 0.4878, 0.6433)
#std: (0.1680, 0.2098, 0.1685)